<h1 align="center">CS340 - Assignment 3</h1>
<h3 align="center">Due Date: 30 April 2017</h3>
<br>
<p style="text-indent: 40px">In this project we are going to use a clustering method(LDA) on the Yelp dataset. The dataset is included in the assignment folder. You should unzip and upload the file to ibm's datascience tool.
</p>


## Topics From Reviews

"LDA is a topic model which infers topics from a collection of text documents."
For the purpose of this assignment we can treat each yelp review as a document and extract two topics from the data by using LDA.

For this assignment you should use ml transformers and ml pipeline. Just for convenince, ReviewsRdd is provided for you. You should convert this rdd to DataFrame and work on it. In order to do this assignment you should revisit PySpark ML documentation many times.

End result should look like this.

<div>
    <img src="http://image.prntscr.com/image/13eb8a01533346c0bb3186dfc5402b73.png" width=200>
</div>

Your solution does not have to be exactly like this, since each time you run the LDA model, it gives you different results. 

<p style="text-indent: 40px"> Note: You should remove stopwords with a feature transformer, the words you should remove are in the stopwords list that we have provided.</p>
<p style="text-indent: 40px">Important: Do not discuss the solution with your friends. <b>Plagiarism</b> will not be tolerated and issue will be referred to the <b>disciplinary committee</b>.</p>


In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')
spark = pyspark.sql.SparkSession(sc)

In [2]:
stopwords = ["i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now","i'll","you'll","he'll","she'll","we'll","they'll","i'd","you'd","he'd","she'd","we'd","they'd","i'm","you're","he's","she's","it's","we're","they're","i've","we've","you've","they've","isn't","aren't","wasn't","weren't","haven't","hasn't","hadn't","don't","doesn't","didn't","won't","wouldn't","shan't","shouldn't","mustn't","can't","couldn't","cannot","could","here's","how's","let's","ought","that's","there's","what's","when's","where's","who's","why's","would"]

# convert all words to lowercase
# remove words that are too long or too short 
reviewsRdd = sc.textFile('/data/Spark/assignment3/100kReviews.txt')\
            .map(lambda line: line.lower())\
            .map(lambda line: " ".join(word for word in line.strip().split() if 25 > len(word) > 4))

In [3]:
reviewsDf = reviewsRdd.map(lambda x: pyspark.sql.types.Row(x)).toDF(['comment'])

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer,StopWordsRemover,CountVectorizer

tokenizer = RegexTokenizer(inputCol="comment", outputCol="words", pattern="\\W")
remover = StopWordsRemover(inputCol="words", outputCol="processed", stopWords=stopwords)
vectorizer = CountVectorizer(inputCol="processed", outputCol="features", vocabSize=10)

pipeline = Pipeline().setStages([tokenizer,remover,vectorizer]).fit(reviewsDf)

In [6]:
from pyspark.mllib.linalg import Vectors

featureRdd = pipeline.transform(reviewsDf).select("features").rdd.map(list)
corpus = featureRdd.map(lambda x: Vectors.dense(x[0])).zipWithIndex().map(lambda x:[x[1],x[0]]).cache()

In [9]:
from pyspark.mllib.clustering import LDA, LDAModel
ldaModel = LDA.train(corpus, k=2)
topicDescription = ldaModel.describeTopics()
wordDictionary = dict(zip(range(len(pipeline.stages[2].vocabulary)),pipeline.stages[2].vocabulary))
for i in range(2):
    print "Topic:"+str(i)
    for index in topicDescription[i][0]:
        print wordDictionary[index]
    print ""

Topic:0
place
great
really
service
staff
always
people
little
friendly
store

Topic:1
place
great
really
service
staff
always
people
little
friendly
store

